In [16]:
from ultralytics import YOLO
import math
import cv2

Calling the pretrained YOLOV8m model

In [17]:
model = YOLO("yolov8m.pt")


In [18]:
classNames = list(model.names.values())
print(classNames)
import random

# Function to generate a random color
def random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Generate a list of 80 different colors
colors_list = [random_color() for _ in range(80)]

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


Video Feed Input

In [19]:
cap = cv2.VideoCapture(0)
cap.set(3,1024)
cap.set(4,576)


True

Main Detection

In [20]:
from PIL import ImageOps, Image
import numpy as np
while True:
    success, img = cap.read()
    if not success:
        break

    # Convert the BGR frame to RGB
    rgb_frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Convert the RGB frame to a PIL Image
    pil_image = Image.fromarray(rgb_frame)
    
    # Apply autocontrast using ImageOps
    ignore_values = list(range(230, 256))
    autocontrast_image = ImageOps.autocontrast(pil_image,ignore=ignore_values)
    
    # Convert the PIL image back to a NumPy array
    autocontrast_frame = np.array(autocontrast_image)
    
    # Convert RGB frame back to BGR
    autocontrast_frame_bgr = cv2.cvtColor(autocontrast_frame, cv2.COLOR_RGB2BGR)
    results = model(autocontrast_frame_bgr, stream=True)

    #coordinates
    for r in results:
        boxes = r.boxes
        
        
        for box in boxes:
            #bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cls = int(box.cls[0])
            #put box in video
            cv2.rectangle(autocontrast_frame_bgr, (x1,y1), (x2,y2), colors_list[cls], 2)
            

            #confidence
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confidence ---> ", confidence)

            #class name
            cls = int(box.cls[0])
            print("Class name --->", classNames[cls])

            #object details
            org = [x1, y1+30]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = colors_list[cls]
            thickness = 2
            cv2.putText(autocontrast_frame_bgr, classNames[cls]+f" {confidence}", org, font, fontScale, color, thickness)

    cv2.imshow('Video', autocontrast_frame_bgr)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 8 persons, 1 tie, 1 chair, 711.6ms
Confidence --->  0.96
Class name ---> person
Confidence --->  0.93
Class name ---> person
Confidence --->  0.91
Class name ---> person
Confidence --->  0.88
Class name ---> tie
Confidence --->  0.7
Class name ---> person
Confidence --->  0.66
Class name ---> person
Confidence --->  0.57
Class name ---> person
Confidence --->  0.34
Class name ---> chair
Confidence --->  0.3
Class name ---> person
Confidence --->  0.28
Class name ---> person
Speed: 4.0ms preprocess, 711.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 tie, 708.8ms
Confidence --->  0.97
Class name ---> person
Confidence --->  0.93
Class name ---> person
Confidence --->  0.89
Class name ---> person
Confidence --->  0.88
Class name ---> tie
Confidence --->  0.57
Class name ---> person
Confidence --->  0.56
Class name ---> person
Confidence --->  0.4
Class name ---> person
Confidence --->  0.37
Class name ---> person
Confidence ---> 